In [1]:
import sys,os
import numpy as np
import pylab as py
# from corelib_AA import JAMLIB

In [2]:
import sys,os
import numpy as np
import cPickle
from scipy.interpolate import RectBivariateSpline
from tools import load,BAR,logo
 
class JAMLIB(object):
  
  def __init__(self,path):
    logo()
    self.setup_alphaS(order='NLO')
    self.load_tables(path)

  # alphaS

  def setup_alphaS(self,order='NLO'):

    if   order=='LO':  self.order=0
    elif order=='NLO': self.order=1

    self.aZ  = 0.118 /(4*np.pi)
    self.mc2 = 1.43**2        
    self.mb2 = 4.3**2
    self.mZ2 = 91.187**2
    self.mt2 = 172.9**2

    self.beta=np.zeros((7,3))
    for Nf in range(3,7): 
      self.beta[Nf,0]=11.0-2.0/3.0*Nf 
      self.beta[Nf,1]=102.-38.0/3.0*Nf 
      self.beta[Nf,2]=2857.0/2.0-5033.0/18.0*Nf+325.0/54.0*Nf**2 

    self.Q20=1.0
    # uses alphaS(mZ)--> backwards evolution
    self.ab=self.evolve_a(self.mZ2,self.aZ,self.mb2,5)
    self.at=self.evolve_a(self.mZ2,self.aZ,self.mt2,5)
    self.ac=self.evolve_a(self.mb2,self.ab,self.mc2,4)
    self.a0=self.evolve_a(self.mc2,self.ac,self.Q20,3)

    # we will store all Q2 values of alphaS 
    self.storage={}

  def get_Nf(self,Q2):
    Nf=3
    if Q2>=(4.*self.mc2): Nf+=1
    if Q2>=(4.*self.mb2): Nf+=1
    if Q2>=(4.*self.mt2): Nf+=1
    return Nf

  def beta_func(self,a,Nf):
    betaf = -self.beta[Nf,0]
    if self.order>=1: betaf+=-a*self.beta[Nf,1]
    if self.order>=2: betaf+=-a*self.beta[Nf,2]
    return betaf*a**2

  def evolve_a(self,Q20,a,Q2,Nf):
    # Runge-Kutta implemented in pegasus  
    LR = np.log(Q2/Q20)/20.0
    for k in range(20):
      XK0 = LR * self.beta_func(a,Nf)
      XK1 = LR * self.beta_func(a + 0.5 * XK0,Nf)
      XK2 = LR * self.beta_func(a + 0.5 * XK1,Nf)
      XK3 = LR * self.beta_func(a + XK2,Nf)
      a+= (XK0 + 2.* XK1 + 2.* XK2 + XK3) * 0.166666666666666
    return a

  def get_a(self,Q2):

    if Q2 in self.storage:
      return self.storage[Q2]
    else:
      if self.mt2<=Q2:
        Q20,a0,Nf=self.mt2,self.at,6
      elif self.mb2<=Q2 and Q2<self.mt2: 
        Q20,a0,Nf=self.mb2,self.ab,5
      elif self.mc2<=Q2 and Q2<self.mb2: 
        Q20,a0,Nf=self.mc2,self.ac,4
      elif Q2<self.mc2:
        Q20,a0,Nf=self.mc2,self.ac,3
      a=self.evolve_a(Q20,a0,Q2,Nf)
      self.storage[Q2]=a
      return a

  def get_alphaS(self,Q2):
    return self.get_a(Q2)*4*np.pi

  # interpolation for xF

  def load_tables(self,path):
    TAB=[]
    F=os.listdir(path)
    self.npos=len(F)
    bar=BAR('loading tables',len(F))
    for f in F:
      tab=load('%s/%s'%(path,f)) 
      X=tab['X']
      Q2=tab['Q2']
      for k in tab:
        if k=='X' or k=='Q2': continue
        for kk in tab[k]:
          tab[k][kk]=RectBivariateSpline(X,Q2,tab[k][kk])
      TAB.append(tab)
      bar.next()
    bar.finish()
    self.TAB=TAB
    self.Xgrid = X
    self.Q2grid = Q2

  def get_XF(self,ipos,dist,flav,x,Q2):
    return self.TAB[ipos][dist][flav](x,Q2)[0,0]
  
  def get_Xgrid(self):
    return self.Xgrid
    
  def get_Q2grid(self):
    return self.Q2grid
  

## Loads and tests the FF

In [3]:
jamFF=JAMLIB('FFpions')

################################################ 
                                                 
     _   _    __  __ _     ___ ____              
    | | / \  |  \/  | |   |_ _| __ )             
 _  | |/ _ \ | |\/| | |    | ||  _ \             
| |_| / ___ \| |  | | |___ | || |_) |            
 \___/_/   \_\_|  |_|_____|___|____/             
                                                 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
                                                 
Authors:                                         
Nobuo Sato   nsato@jlab.org                      
Jake Ethier                                      
Wally Melnitchouk                                
Alberto Accardi                                  
################################################ 
loading tables [99%]


In [51]:
def testjam(x,Q2,jamhadron):

  print 'alphaS     = ',jamFF.get_alphaS(Q2)
  print 'no. of posteriors = ',jamFF.npos
  print 'xFF(ipos=0) = ',jamFF.get_XF(0,jamhadron,'up',x,Q2)

  xgrid = jamFF.get_Xgrid()
  xmin = np.min(xgrid)
  xmax = np.max(xgrid)
  print 'Xmin,Xmax: ', xmin,xmax
  print xgrid 

  Q2grid = jamFF.get_Q2grid()
  Q2min = np.min(Q2grid)
  Q2max = np.max(Q2grid)
  print 'Q2min,Q2max: ', Q2min,Q2max
  print Q2grid

In [52]:
testjam(0.1,10.,'FFpion')

alphaS     =  0.24654958796
no. of posteriors =  200
xFF(ipos=0) =  0.34776866759
Xmin,Xmax:  1e-06 1.0
[  1.00000000e-06   1.26485522e-06   1.59985872e-06   2.02358965e-06
   2.55954792e-06   3.23745754e-06   4.09491506e-06   5.17947468e-06
   6.55128557e-06   8.28642773e-06   1.04811313e-05   1.32571137e-05
   1.67683294e-05   2.12095089e-05   2.68269580e-05   3.39322177e-05
   4.29193426e-05   5.42867544e-05   6.86648845e-05   8.68511374e-05
   1.09854114e-04   1.38949549e-04   1.75751062e-04   2.22299648e-04
   2.81176870e-04   3.55648031e-04   4.49843267e-04   5.68986603e-04
   7.19685673e-04   9.10298178e-04   1.15139540e-03   1.45634848e-03
   1.84206997e-03   2.32995181e-03   2.94705170e-03   3.72759372e-03
   4.71486636e-03   5.96362332e-03   7.54312006e-03   9.54095476e-03
   1.20679264e-02   1.52641797e-02   1.93069773e-02   2.44205309e-02
   3.08884360e-02   3.90693994e-02   4.94171336e-02   6.25055193e-02
   7.90604321e-02   1.00000000e-01   1.18000000e-01   1.36000000e-01

## Grid setup example

The following is user-supplied information - it changes with each LHAPDF set one wants to create

In [35]:
# Select JAM fit and LHAPDF set name
jamfit = "FFPIONS"
setname = "JAM16FF_pi"   # name of the set as it should appear in LHAPDF
hadron = "FFpion" 

# User-supplied  information for .info file headers
desc = 'JAMFF16 pion fragmentation functions. Returns x*FF(x). \
Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-200 => FF posteriors.'
auth = "[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A"
ref = "arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org"
ver = -1   # -1=beta; 1,2,3, ... official version, and updated/debugged ones if needed
  # index = nnxxx
  # nn (or nnn): assigned by Andy [LHAPDF]; for example 16=JAM fits - this to be discussed with Andy
  # xxx (or xx): up to us --> I would suggest 000-002 for JAM15; 003-004 for JAM16 FF; and so on as needed
index = 16003

The following collects information common to all (JAM16) fits, used to write the .info and .dat files. It could be put into a function, but for now I am passing the information as global variables. 

In [59]:
# COLLECTS INFO ON JAM FIT

# Parent "particle"
  # "particle" is the hadron to which the PDFs belong, see
  # http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
  # For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
  # E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
  # For nuclei, use LZZZAAAI, see point 14 in PDG standard:
  # L nonzero is for hypernuclei, I represent isomer levels with
  # I = 0 the ground state, and Z and A should be self-explanatory.
particle = 2212  #  2212=proton

# QCD setup
ordQCD =1  # 1: NLO
nfl = 5
#           Usual PDFs, with q/qbar separation
#flavors = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
#partons = {'bbar','cbar','sbar','ubar','dbar','d','u','s','c','b',21:'g'}
#          DIS/SIA-only analysis --> only q+=q+ + q- is fitted
flavors = [1, 2, 3, 4, 5, 21]
partons = ['dp','up','sp','cp','bp','g']


# This is for JAM FF or PDF, for others needs to change
errortype = 'MC'      # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = jamFF.npos # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3             # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.
mc = jamFF.mc2**0.5
mb = jamFF.mb2**0.5
mt = jamFF.mt2**0.5
mZ = jamFF.mZ2**0.5

## Writes .info file

In [32]:
def writeinfo():
  H=[]
  H.append('SetDesc: '+desc)
  H.append('SetIndex: %i'%index)
  H.append('Authors: '+auth)
  H.append('Reference: '+ref)
  H.append('Format: lhagrid1')
  H.append('DataVersion: %i'%ver)

  H.append('Particle: %i'%particle)
  H.append('Flavors: '+str(flavors))
  H.append('OrderQCD: %i'%ordQCD)
  H.append('ForcePositive: 1')
  flavscheme = 'variable' # used for any variable flavor number scheme, massive or not
  H.append('FlavorScheme: '+flavscheme)
  H.append('NumFlavors: %i'%nfl)

  if errortype=='MC':
    nsets = nsamples+1   # number of posteriors + average
    errtype = 'replicas'  # LHAPDF-speak for MC method
  elif errortype=='HESSIAN': 
    nsets = 2*nsamples+2  # 2*eigenvalues + central fit
    errtype = 'hessian'
  H.append('NumMembers: %i'%nsets)
  H.append('ErrorType: '+errtype)
  H.append('ErrorConfLevel: %f'%cl)
  H.append('XMin: %e'%xmin)
  H.append('XMax: %e'%xmax)
  H.append('QMin: %e'%Qmin)
  H.append('QMax: %e'%Qmax)

  H.append('MZ: %f'%mZ)
  H.append('MUp: %f'%mu)
  H.append('MDown: %f'%md)
  H.append('MStrange: %f'%ms)
  H.append('MCharm: %f'%mc)
  H.append('MBottom: %f'%mb)
  H.append('MTop: %f'%mt)

  H.append('AlphaS_MZ: %f'%jamFF.get_alphaS(mZ**2))
  H.append('AlphaS_OrderQCD: %f'%ordQCD)
  H.append('AlphaS_Type: ipol')
  H.append('AlphaS_Qs: '+str(Qgrid.tolist()))
  alphas = []
  for Q in Qgrid:
    alphas.append(jamFF.get_alphaS(Q**2))
  H.append('AlphaS_Vals: '+str(alphas))
  H.append('AlphaS_Lambda4: %f'%jamFF.get_alphaS(mc**2))
  H.append('AlphaS_Lambda5: %f'%jamFF.get_alphaS(mb**2))

  for l in H: print l 

Test of .info file creation

In [61]:
writeinfo()

SetDesc: JAMFF16 pion fragmentation functions. Returns x*FF(x). Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. mem=0 => average of posteriors; mem=1-200 => FF posteriors.
SetIndex: 16003
Authors: [JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A
Reference: arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org
Format: lhagrid1
DataVersion: -1
Particle: 2212
Flavors: [1, 2, 3, 4, 5, 21]
OrderQCD: 1
ForcePositive: 1
FlavorScheme: variable
NumFlavors: 5
NumMembers: 201
ErrorType: replicas
ErrorConfLevel: 68.300000
XMin: 1.000000e-06
XMax: 1.000000e+00
QMin: 1.000000e+00
QMax: 1.000000e+05
MZ: 91.187000
MUp: 0.000000
MDown: 0.000000
MStrange: 0.000000
MCharm: 1.430000
MBottom: 4.300000
MTop: 172.900000
AlphaS_MZ: 0.118000
AlphaS_OrderQCD: 1.000000
AlphaS_Type: ipol
AlphaS_Qs: [1.0, 1.4873521072935114, 2.2122162910704493, 3.2903445623126686, 4.893900918477494, 7.278953843983151, 10.826367338740546, 16.102620275609393, 23.95026619987486, 3

## Write a single .dat file

In [73]:
def writedat(ipos,hadron):
  
  if ipos==0: 
    type='central'
  else:
    type='replica'

  H = []
  H.append('PdfType: '+type)
  H.append('Format: lhagrid1')
  H.append('---')  
  
  for l in H: print l
  
  xvals = ["%10.4e" % x for x in xgrid]
  print ' '.join(xvals)
  Qvals = ["%10.4e" % Q for Q in Qgrid]
  print ' '.join(Qvals)
  flavs = ["%2i" % f for f in flavors]
  print ' '.join(flavs)
  
  npart = len(partons)
  
  full_dist = []
  ix = 0
  iQ = 0
  il = 0
  for x in xgrid: 
    for Q in Qgrid:
      if ipos==0:  # Avreraged PDF
        FF = np.array([0 for n in range(npart)])
        for i in range(len(partons)): 
          FF = FF + np.array([jamFF.get_XF(i,hadron,f,x,Q**2) for f in partons])
        FF = FF/nsamples
      else:   # ipos-th posterior 
        FF = [jamFF.get_XF(ipos-1,hadron,f,x,Q**2) for f in partons]   # <--------- NOTE: for now x*FF(x), Shall we distribute FF(x) as for other PDFs ??  
        #if Q < mb: FF[0] = FF[9] = 0
        #if Q < mc: FF[1] = FF[8] = 0
        if Q < mb: FF[4] = 0
        if Q < mc: FF[3] = 0
      #FF.append([x,Q])
      full_dist.append(FF)
  template = ''.join(['{:12.5e} ' for n in range(npart)])    # <----- how many sig digits ?
  for l in full_dist: 
    print template.format(l[0],l[1],l[2],l[3],l[4],l[5])  # <--- horrible but functional (for now) python 
    #print template.format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10])  # <--- horrible but functional (for now) python 
    #print template.format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]),":",l[11] 
        
 

Test of .dat file output

In [74]:
print partons

['dp', 'up', 'sp', 'cp', 'bp', 'g']


In [75]:
hadron = 'FFkaon'
writedat(1,hadron)

PdfType: replica
Format: lhagrid1
---
1.0000e-06 1.2649e-06 1.5999e-06 2.0236e-06 2.5595e-06 3.2375e-06 4.0949e-06 5.1795e-06 6.5513e-06 8.2864e-06 1.0481e-05 1.3257e-05 1.6768e-05 2.1210e-05 2.6827e-05 3.3932e-05 4.2919e-05 5.4287e-05 6.8665e-05 8.6851e-05 1.0985e-04 1.3895e-04 1.7575e-04 2.2230e-04 2.8118e-04 3.5565e-04 4.4984e-04 5.6899e-04 7.1969e-04 9.1030e-04 1.1514e-03 1.4563e-03 1.8421e-03 2.3300e-03 2.9471e-03 3.7276e-03 4.7149e-03 5.9636e-03 7.5431e-03 9.5410e-03 1.2068e-02 1.5264e-02 1.9307e-02 2.4421e-02 3.0888e-02 3.9069e-02 4.9417e-02 6.2506e-02 7.9060e-02 1.0000e-01 1.1800e-01 1.3600e-01 1.5400e-01 1.7200e-01 1.9000e-01 2.0800e-01 2.2600e-01 2.4400e-01 2.6200e-01 2.8000e-01 2.9800e-01 3.1600e-01 3.3400e-01 3.5200e-01 3.7000e-01 3.8800e-01 4.0600e-01 4.2400e-01 4.4200e-01 4.6000e-01 4.7800e-01 4.9600e-01 5.1400e-01 5.3200e-01 5.5000e-01 5.6800e-01 5.8600e-01 6.0400e-01 6.2200e-01 6.4000e-01 6.5800e-01 6.7600e-01 6.9400e-01 7.1200e-01 7.3000e-01 7.4800e-01 7.6600e-01 7.840

## Writes all the .info and .dat files in one folder

In [39]:
def create_LHAPDF_grid(setname,hadron):
  
  start = os.getcwd()

  # Creates and navigates to the right folder
  pre = '../LHAPDF_grids'
  if not os.path.exists(pre):
      os.makedirs(pre)
  os.chdir(pre)
  if not os.path.exists(setname):
      os.makedirs(setname)
  os.chdir(setname)

  # Writes the .info file
  stdout = sys.stdout 
  f = open(setname+'.info','w')   
  sys.stdout = f    # <---------- very fragile solution, needs improvement
  writeinfo()
  f.close()
  sys.stdout=stdout

  for igrid in range(nsamples+1):
    gridname = '{}_{:04d}.dat'.format(setname,igrid)
    f = open(gridname,'w')
    sys.stdout = f   # <---------- very fragile solution, needs improvement
    writedat(igrid,hadron)
    f.close()
    sys.stdout=stdout
  
  # goes back to the initial folder
  os.chdir(start)



# GRID PRODUCTION

In [91]:
# Select JAM analysis
jam_analysis = "JAM16/FFpions"
jamFF=JAMLIB(jam_analysis)

# Selects LHAPDF set name for given JAM hadron
setname = "JAM16FF_pi"   # name of teh set as it should appear in LHAPDF
jamhadron = "FFpion" 

# User-supplied  information for .info file headers               # <-------------  CHECK x*FF or FF for LHAPDF comvention
desc = 'JAMFF16 fragmentation functions into pi+ mesons (identical to pi- by charge conjugation). Returns x*FF(x). \
Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. \
mem=0 => average of posteriors; mem=1-200 => FF posteriors.'
auth = "[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A"
ref = "arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org"
ver = -1   # -1=beta; 1,2,3, ... official version, and updated/debugged ones if needed
  # index = nnxxx
  # nn (or nnn): assigned by Andy [LHAPDF]; for example 16=JAM fits - this to be discussed with Andy
  # xxx (or xx): up to us --> I would suggest 000-002 for JAM15; 003-004 for JAM16 FF; and so on as needed
index = 16003

################################################ 
                                                 
     _   _    __  __ _     ___ ____              
    | | / \  |  \/  | |   |_ _| __ )             
 _  | |/ _ \ | |\/| | |    | ||  _ \             
| |_| / ___ \| |  | | |___ | || |_) |            
 \___/_/   \_\_|  |_|_____|___|____/             
                                                 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
                                                 
Authors:                                         
Nobuo Sato   nsato@jlab.org                      
Jake Ethier                                      
Wally Melnitchouk                                
Alberto Accardi                                  
################################################ 
loading tables [99%]


In [92]:
testjam(0.1,10.,jamhadron)

alphaS     =  0.24654958796
no. of posteriors =  200
xFF(ipos=0) =  1.77777274434
Xmin,Xmax:  0.05 1.0
[ 0.05        0.05959596  0.06919192  0.07878788  0.08838384  0.0979798
  0.10757576  0.11717172  0.12676768  0.13636364  0.1459596   0.15555556
  0.16515152  0.17474747  0.18434343  0.19393939  0.20353535  0.21313131
  0.22272727  0.23232323  0.24191919  0.25151515  0.26111111  0.27070707
  0.28030303  0.28989899  0.29949495  0.30909091  0.31868687  0.32828283
  0.33787879  0.34747475  0.35707071  0.36666667  0.37626263  0.38585859
  0.39545455  0.40505051  0.41464646  0.42424242  0.43383838  0.44343434
  0.4530303   0.46262626  0.47222222  0.48181818  0.49141414  0.5010101
  0.51060606  0.52020202  0.52979798  0.53939394  0.5489899   0.55858586
  0.56818182  0.57777778  0.58737374  0.5969697   0.60656566  0.61616162
  0.62575758  0.63535354  0.64494949  0.65454545  0.66414141  0.67373737
  0.68333333  0.69292929  0.70252525  0.71212121  0.72171717  0.73131313
  0.74090909  0.7505050

In [93]:
# COLLECTS INFO ON JAM FIT

# Parent "particle"
  # "particle" is the hadron to which the PDFs belong, see
  # http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
  # For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
  # E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
  # For nuclei, use LZZZAAAI, see point 14 in PDG standard:
  # L nonzero is for hypernuclei, I represent isomer levels with
  # I = 0 the ground state, and Z and A should be self-explanatory.
particle = 2212  #  2212=proton

# QCD setup
ordQCD =1  # 1: NLO
nfl = 5
#           Usual PDFs, with q/qbar separation
#           bb  cb  sb  ub  db  d  u  s  c  b  g
#flavors = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
#          DIS/SIA-only analysis --> only q+=q+ + q- is fitted
#          d+ u+ s+ c+ b+  g
flavors = [1, 2, 3, 4, 5, 21]
partons = ['dp','up','sp','cp','bp','g']

# This is for JAM FF or PDF, for others needs to change
errortype = 'MC'      # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = jamFF.npos # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3             # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.
mc = jamFF.mc2**0.5
mb = jamFF.mb2**0.5
mt = jamFF.mt2**0.5
mZ = jamFF.mZ2**0.5

In [94]:
writeinfo()

SetDesc: JAMFF16 fragmentation functions into pi+ mesons (identical to pi- by charge conjugation). Returns x*FF(x). Flavors 1-5: q+ == q+qbar with q=d,u,s,c,b; Flavor 21: gluon. mem=0 => average of posteriors; mem=1-200 => FF posteriors.
SetIndex: 16003
Authors: [JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A
Reference: arXiv:1609.00899 - e-mail accardi@jlab.org / sato@jlab.org
Format: lhagrid1
DataVersion: -1
Particle: 2212
Flavors: [1, 2, 3, 4, 5, 21]
OrderQCD: 1
ForcePositive: 1
FlavorScheme: variable
NumFlavors: 5
NumMembers: 201
ErrorType: replicas
ErrorConfLevel: 68.300000
XMin: 5.000000e-02
XMax: 1.000000e+00
QMin: 1.000000e+00
QMax: 1.000000e+05
MZ: 91.187000
MUp: 0.000000
MDown: 0.000000
MStrange: 0.000000
MCharm: 1.430000
MBottom: 4.300000
MTop: 172.900000
AlphaS_MZ: 0.118000
AlphaS_OrderQCD: 1.000000
AlphaS_Type: ipol
AlphaS_Qs: [1.0, 1.4873521072935114, 2.2122162910704493, 3.2903445623126686, 4.893900918477494, 7.278953843983151, 10.82636

In [95]:
writedat(1,jamhadron)

PdfType: replica
Format: lhagrid1
---
5.0000e-02 5.9596e-02 6.9192e-02 7.8788e-02 8.8384e-02 9.7980e-02 1.0758e-01 1.1717e-01 1.2677e-01 1.3636e-01 1.4596e-01 1.5556e-01 1.6515e-01 1.7475e-01 1.8434e-01 1.9394e-01 2.0354e-01 2.1313e-01 2.2273e-01 2.3232e-01 2.4192e-01 2.5152e-01 2.6111e-01 2.7071e-01 2.8030e-01 2.8990e-01 2.9949e-01 3.0909e-01 3.1869e-01 3.2828e-01 3.3788e-01 3.4747e-01 3.5707e-01 3.6667e-01 3.7626e-01 3.8586e-01 3.9545e-01 4.0505e-01 4.1465e-01 4.2424e-01 4.3384e-01 4.4343e-01 4.5303e-01 4.6263e-01 4.7222e-01 4.8182e-01 4.9141e-01 5.0101e-01 5.1061e-01 5.2020e-01 5.2980e-01 5.3939e-01 5.4899e-01 5.5859e-01 5.6818e-01 5.7778e-01 5.8737e-01 5.9697e-01 6.0657e-01 6.1616e-01 6.2576e-01 6.3535e-01 6.4495e-01 6.5455e-01 6.6414e-01 6.7374e-01 6.8333e-01 6.9293e-01 7.0253e-01 7.1212e-01 7.2172e-01 7.3131e-01 7.4091e-01 7.5051e-01 7.6010e-01 7.6970e-01 7.7929e-01 7.8889e-01 7.9848e-01 8.0808e-01 8.1768e-01 8.2727e-01 8.3687e-01 8.4646e-01 8.5606e-01 8.6566e-01 8.7525e-01 8.848

In [96]:
# CREATE THE GRIDS

create_LHAPDF_grid(setname,jamhadron)